# **삼성전자 5년 상향**

In [ ]:
### 크롤링에 필요한 모듈 불러오기
import requests
from bs4 import BeautifulSoup
import time
import pandas as pd
import re

## 1 Page

 ✅ 참고 사이트

 https://gils-lab.tistory.com/14

### URL

In [ ]:
### requests.get()로 페이지 열기 -> BeautifulSoup의 입력
url = "http://hkconsensus.hankyung.com/apps.analysis/analysis.list?skinType=stock_good&sdate=2018-06-05&edate=2023-06-05&search_value=&search_text=%BB%EF%BC%BA%C0%FC%C0%DA&order_type=&pagenum=80"

# headers를 입력해야 데이터가 리턴됨
html = requests.get(url, headers={'User-Agent':'Gils'}).content
soup = BeautifulSoup(html, 'lxml')

In [ ]:
### soup의 find로 table_style01인 div 요소 찾기
'''
find() : 조건 만족하는 첫 번째 요소 반환하고, find_all함수는 모든 요소를 리스트 형태로 반환
'''
soup.find("div",{"class":"table_style01"})

<div class="table_style01">
<table>
<colgroup>
<col width="98px"/>
<col width="*"/>
<col width="135px"/>
<col width="146px"/>
<col width="93px"/>
<col width="93px"/>
</colgroup>
<thead>
<tr>
<th class="first" rowspan="2"><a href="javascript:_Order_Link(10000000,1000000,'/apps.analysis/analysis.list?skinType=stock_good&amp;sdate=2018-06-05&amp;edate=2023-06-05&amp;search_value=&amp;search_text=삼성전자&amp;pagenum=80&amp;order_type=&amp;now_page=1')">작성일 <img alt="" src="/images/bg_table01.gif"/></a></th>
<th rowspan="2"><a href="javascript:_Order_Link(10000000,10000,'/apps.analysis/analysis.list?skinType=stock_good&amp;sdate=2018-06-05&amp;edate=2023-06-05&amp;search_value=&amp;search_text=삼성전자&amp;pagenum=80&amp;order_type=&amp;now_page=1')">제목 <img alt="" src="/images/bg_table01.gif"/></a></th>
<th rowspan="2"><a href="javascript:_Order_Link(10000000,1000,'/apps.analysis/analysis.list?skinType=stock_good&amp;sdate=2018-06-05&amp;edate=2023-06-05&amp;search_value=&amp;search_text=삼성전자&amp

### 데이터 파악

In [ ]:
### 출력값에서 <table> 요소 찾은 것을 table에 저장하기
table = soup.find("div",{"class":"table_style01"}).find("table")

In [ ]:
### 테이블에서 모든 행을 찾아서 정리하기
'''
table 요소에서 행이 모두 tr로 정리되어 있음 확인
'''

# 첫 행의 구성 살펴보기
print(table.find("tr"))

<tr>
<th class="first" rowspan="2"><a href="javascript:_Order_Link(10000000,1000000,'/apps.analysis/analysis.list?skinType=stock_good&amp;sdate=2018-06-05&amp;edate=2023-06-05&amp;search_value=&amp;search_text=삼성전자&amp;pagenum=80&amp;order_type=&amp;now_page=1')">작성일 <img alt="" src="/images/bg_table01.gif"/></a></th>
<th rowspan="2"><a href="javascript:_Order_Link(10000000,10000,'/apps.analysis/analysis.list?skinType=stock_good&amp;sdate=2018-06-05&amp;edate=2023-06-05&amp;search_value=&amp;search_text=삼성전자&amp;pagenum=80&amp;order_type=&amp;now_page=1')">제목 <img alt="" src="/images/bg_table01.gif"/></a></th>
<th rowspan="2"><a href="javascript:_Order_Link(10000000,1000,'/apps.analysis/analysis.list?skinType=stock_good&amp;sdate=2018-06-05&amp;edate=2023-06-05&amp;search_value=&amp;search_text=삼성전자&amp;pagenum=80&amp;order_type=&amp;now_page=1')">작성자 <img alt="" src="/images/bg_table01.gif"/></a></th>
<th rowspan="2"><a href="javascript:_Order_Link(10000000,100,'/apps.analysis/analysi

In [ ]:
# 첫 행의 세번째 행 살펴보기
table.find_all("tr")[1]

<tr>
<th>현재목표</th>
<th>이전목표</th>
</tr>

In [ ]:
# 첫 행의 세번째 행 살펴보기
table.find_all("tr")[2]

<tr class="first">
<td class="first txt_number">2023-04-10</td>
<td class="text_l">
<a href="javascript:void(0);" onmouseout="$('#content_620518').hide();" onmouseover="$('#content_620518').show();">삼성전자(005930) 업황이 나쁠수록 감산은 확대된다.</a>
<div class="layerPop">
<div class="pop01 disNone" id="content_620518">
<strong>삼성전자(005930) 업황이 나쁠수록 감산은 확대된다.</strong>
<ul>
<li>업황이 나쁠수록 감산은 확대된다.
</li><li></li>
</ul>
</div>
</div>
</td>
<td>송명섭</td>
<td>하이투자증권</td>
<td class="text_r txt_number">83,400</td>
<td class="text_r txt_number">75,000</td>
</tr>

In [ ]:
# 첫 행의 세번째 행 위치의 텍스트만 반환하기
table.find_all("tr")[2].text

'\n2023-04-10\n\n삼성전자(005930) 업황이 나쁠수록 감산은 확대된다.\n\n\n삼성전자(005930) 업황이 나쁠수록 감산은 확대된다.\n\n업황이 나쁠수록 감산은 확대된다.\r\n\n\n\n\n송명섭\n하이투자증권\n83,400\n75,000\n'

### 종목코드와 종목명 크롤링

In [ ]:
!pip install finance-datareader

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
### FinanceDataReader 사용
import FinanceDataReader as fdr

### 종목코드와 종목명 가져오기
stocks = fdr.StockListing("KRX")
stocks.head()

,Code,ISU_CD,Name,Market,Dept,Close,ChangeCode,Changes,ChagesRatio,Open,High,Low,Volume,Amount,Marcap,Stocks,MarketId
0,005930,KR7005930003,삼성전자,KOSPI,,71900,2,-100,-0.14,72100,72200,71100,12453599,891989441300,428630387090000,5969782550,STK
1,373220,KR7373220003,LG에너지솔루션,KOSPI,,589000,2,-18000,-2.97,610000,614000,588000,297005,178257624000,138294000000000,234000000,STK
2,000660,KR7000660001,SK하이닉스,KOSPI,,118800,2,-700,-0.59,118800,119700,118100,3134981,372404707200,86559481198500,728002365,STK
3,207940,KR7207940008,삼성바이오로직스,KOSPI,,784000,2,-2000,-0.25,788000,789000,783000,47189,37065447000,55871590000000,71174000,STK
4,051910,KR7051910008,LG화학,KOSPI,,719000,2,-17000,-2.31,736000,739000,716000,262708,191073706000,50614709931000,70592343,STK


✅ 종목명과 코드가 없는 제목이면 [None] 리턴 -> 추후에 데이터 추가하지 않도록 처리

In [ ]:
stocks['Code'] = stocks['Code'].astype(str)
def remove_noise_title(title):
  in_code = ''
  in_name = ''

  # KRX 종목명과 코드에 있는 제목만 넣는 코드
  for code, name in stocks[['Code','Name']].values:
    if code in title and name in title:
      in_code = code
      in_name = name

  # 한글, 영어, 숫자 외 노이즈 제거
  clean_title = re.sub('[^A-Za-z0-9가-힣]', ' ', title)

  # 기업명 코드 수정
  clean_title = clean_title.replace(in_code, ' ')
  clean_title = clean_title.replace(in_name, ' ')
  while ' ' * 2 in clean_title:
    clean_title = clean_title.replace(' '*2, ' ')

  # 기업명이 없는 제목이면, 데이터 추가 X
  if in_name == '':
    return [None]
  else:
    return [in_name, in_code, clean_title]

### 모든 값 저장

✅ 작성일, 제목, 적정가격, 투자의견, 작성자, 제공출처 <- 6개 셀 순환

  종목명 및 코드 <- 1번째 셀

In [ ]:
data = []

# 첫번째 행부터 순회시키기
for tr in table.find_all("tr")[2:]:
  record = []
  # 종목 코드가 없는 경우 오류 발생 -> try except 구문으로 넘기는 작업 추가
  try:
    for i, td in enumerate(tr.find_all("td")[:6]): # 6번째 셀까지
      if i == 1:
        # remove_noise_title의 출려과 이어 붙이기

        record += remove_noise_title(td.text.split('\n\n')[1])
      # 노이즈가 있는 세번째 셀만 따로 처리
      elif i ==3 :
        record.append(td.text.replace(" ","").replace("\r","").replace("\n",""))
      # 1번째 셀이 아닌 경우
      else:
        record.append(td.text)  # <- 셀의 텍스트 값 그대로 입력

    data.append(record)
  except:
    pass

### Data -> DataFrame

In [ ]:
# df : 제목, 라벨링
samsung_up_1p = pd.DataFrame(data, columns = ["작성일","종목명","종목코드","제목","작성자",
                              "작성기관","현재목표","이전목표"])

In [ ]:
samsung_up_1p

,작성일,종목명,종목코드,제목,작성자,작성기관,현재목표,이전목표
0,2023-04-10,삼성전자,005930,업황이 나쁠수록 감산은 확대된다,송명섭,하이투자증권,"83,400","75,000"
1,2023-03-15,삼성전자,005930,뻔하지만 당연한 최고의 선택,위민복,대신증권,"80,000","65,000"
2,2023-01-09,삼성전자,005930,4Q22 잠정실적 리뷰,김광진,한화투자증권,"82,000","73,000"
3,2022-11-21,삼성전자,005930,경쟁력 격차 복구 여부에 주목 저점 분할 매수 권고,송명섭,하이투자증권,"80,000","72,000"
4,2022-07-29,삼성전자,005930,시험은 어렵지만 변별력은 높아진다,이승우,유진투자증권,"83,000","79,000"
...,...,...,...,...,...,...,...,...
75,2020-01-14,삼성전자,005930,새로운 10 년을 준비 중,이순학,한화투자증권,"70,000","64,000"
76,2020-01-13,삼성전자,005930,마지막 한고비 이후 점진 회복,김선우,메리츠종금증권,"70,000","63,000"
77,2020-01-09,삼성전자,005930,4분기 기대 이상,김운호,IBK투자증권,"68,000","57,000"
78,2020-01-09,삼성전자,005930,1Q20 메모리 가격 및 전사 영업이익 전망치 상향,"박유약, 김소원",키움증권,"69,000","65,000"


In [ ]:
samsung_up_1p['제목'] = samsung_up_1p['종목명']+samsung_up_1p['제목']

### DataFrame 라벨링

In [ ]:
samsung_up_1p['label'] = 1

In [ ]:
samsung_up_1p.drop(labels=['작성자', '작성기관','현재목표','이전목표'], axis = 1,inplace=True)

In [ ]:
samsung_up_1p

,작성일,종목명,종목코드,제목,label
0,2023-04-10,삼성전자,005930,삼성전자 업황이 나쁠수록 감산은 확대된다,1
1,2023-03-15,삼성전자,005930,삼성전자 뻔하지만 당연한 최고의 선택,1
2,2023-01-09,삼성전자,005930,삼성전자 4Q22 잠정실적 리뷰,1
3,2022-11-21,삼성전자,005930,삼성전자 경쟁력 격차 복구 여부에 주목 저점 분할 매수 권고,1
4,2022-07-29,삼성전자,005930,삼성전자 시험은 어렵지만 변별력은 높아진다,1
...,...,...,...,...,...
75,2020-01-14,삼성전자,005930,삼성전자 새로운 10 년을 준비 중,1
76,2020-01-13,삼성전자,005930,삼성전자 마지막 한고비 이후 점진 회복,1
77,2020-01-09,삼성전자,005930,삼성전자 4분기 기대 이상,1
78,2020-01-09,삼성전자,005930,삼성전자 1Q20 메모리 가격 및 전사 영업이익 전망치 상향,1


## 2 Page

In [ ]:
### requests.get()로 페이지 열기 -> BeautifulSoup의 입력
url = "http://hkconsensus.hankyung.com/apps.analysis/analysis.list?skinType=stock_good&sdate=2018-06-05&edate=2023-06-05&search_value=&search_text=%BB%EF%BC%BA%C0%FC%C0%DA&pagenum=80&order_type=&now_page=2"

# headers를 입력해야 데이터가 리턴됨
html = requests.get(url, headers={'User-Agent':'Gils'}).content
soup = BeautifulSoup(html, 'lxml')

In [ ]:
### soup의 find로 table_style01인 div 요소 찾기
'''
find() : 조건 만족하는 첫 번째 요소 반환하고, find_all함수는 모든 요소를 리스트 형태로 반환
'''

'\nfind() : 조건 만족하는 첫 번째 요소 반환하고, find_all함수는 모든 요소를 리스트 형태로 반환\n'

In [ ]:
### 출력값에서 <table> 요소 찾은 것을 table에 저장하기
table = soup.find("div",{"class":"table_style01"}).find("table")

In [ ]:
data = []

# 첫번째 행부터 순회시키기
for tr in table.find_all("tr")[2:]:
  record = []
  # 종목 코드가 없는 경우 오류 발생 -> try except 구문으로 넘기는 작업 추가
  try:
    for i, td in enumerate(tr.find_all("td")[:6]): # 6번째 셀까지
      if i == 1:
        # remove_noise_title의 출려과 이어 붙이기

        record += remove_noise_title(td.text.split('\n\n')[1])
      # 노이즈가 있는 세번째 셀만 따로 처리
      elif i ==3 :
        record.append(td.text.replace(" ","").replace("\r","").replace("\n",""))
      # 1번째 셀이 아닌 경우
      else:
        record.append(td.text)  # <- 셀의 텍스트 값 그대로 입력

    data.append(record)
  except:
    pass

### Data -> DataFrame

In [ ]:
# df : 제목, 라벨링
samsung_up_2p = pd.DataFrame(data, columns = ["작성일","종목명","종목코드","제목","작성자",
                              "작성기관","현재목표","이전목표"])

In [ ]:
samsung_up_2p

,작성일,종목명,종목코드,제목,작성자,작성기관,현재목표,이전목표
0,2020-01-06,삼성전자,005930,2020년 주가 전망,이승우,유진투자증권,"65,000","62,000"
1,2020-01-02,삼성전자,005930,It s bliss,"김경민, 김록호",하나금융투자,"63,000","60,000"
2,2020-01-02,삼성전자,005930,새로운 서막이 될 1분기 실적,"노근창, 박찬호",현대차투자증권,"71,000","61,000"
3,2019-12-23,삼성전자,005930,반도체가 좋으니까,"이수빈,박강호",대신증권,"70,000","64,000"
4,2019-12-11,삼성전자,005930,4Q19 반도체 호조 디스플레이 부진 예상,박유악,키움증권,"65,000","63,000"
5,2019-11-28,삼성전자,005930,2020 년은 나의 해,이수빈,대신증권,"64,000","59,000"
6,2019-11-25,삼성전자,005930,4Q19에 잠시 쉬어갈 수 있으나,송명섭,하이투자증권,"60,000","56,000"
7,2019-11-21,삼성전자,005930,새로운 기술 트렌드는 우리가 이끈다,이순학,한화투자증권,"64,000","58,000"
8,2019-11-19,삼성전자,005930,한고비넘어두개의선물,김선우,메리츠종금증권,"63,000","60,000"
9,2019-11-19,삼성전자,005930,더 나아진 사업 포트폴리오 새로운 도약이 기대된다,김영우,SK증권,"68,000","60,000"


In [ ]:
samsung_up_2p['제목'] = samsung_up_2p['종목명']+samsung_up_2p['제목']

### DataFrame 라벨링

In [ ]:
samsung_up_2p['label'] = 1

In [ ]:
samsung_up_2p.drop(labels=['작성자', '작성기관','현재목표','이전목표'], axis = 1,inplace=True)

In [ ]:
samsung_up_2p

,작성일,종목명,종목코드,제목,label
0,2020-01-06,삼성전자,005930,삼성전자 2020년 주가 전망,1
1,2020-01-02,삼성전자,005930,삼성전자 It s bliss,1
2,2020-01-02,삼성전자,005930,삼성전자 새로운 서막이 될 1분기 실적,1
3,2019-12-23,삼성전자,005930,삼성전자 반도체가 좋으니까,1
4,2019-12-11,삼성전자,005930,삼성전자 4Q19 반도체 호조 디스플레이 부진 예상,1
5,2019-11-28,삼성전자,005930,삼성전자 2020 년은 나의 해,1
6,2019-11-25,삼성전자,005930,삼성전자 4Q19에 잠시 쉬어갈 수 있으나,1
7,2019-11-21,삼성전자,005930,삼성전자 새로운 기술 트렌드는 우리가 이끈다,1
8,2019-11-19,삼성전자,005930,삼성전자 한고비넘어두개의선물,1
9,2019-11-19,삼성전자,005930,삼성전자 더 나아진 사업 포트폴리오 새로운 도약이 기대된다,1


# **삼성전자 5년 하향**

## 1 Page

In [ ]:
### requests.get()로 페이지 열기 -> BeautifulSoup의 입력
url = "http://hkconsensus.hankyung.com/apps.analysis/analysis.list?skinType=stock_bad&sdate=2018-06-05&edate=2023-06-05&search_value=&search_text=%BB%EF%BC%BA%C0%FC%C0%DA&order_type=&pagenum=80"

# headers를 입력해야 데이터가 리턴됨
html = requests.get(url, headers={'User-Agent':'Gils'}).content
soup = BeautifulSoup(html, 'lxml')

In [ ]:
### soup의 find로 table_style01인 div 요소 찾기
'''
find() : 조건 만족하는 첫 번째 요소 반환하고, find_all함수는 모든 요소를 리스트 형태로 반환
'''

'\nfind() : 조건 만족하는 첫 번째 요소 반환하고, find_all함수는 모든 요소를 리스트 형태로 반환\n'

In [ ]:
### 출력값에서 <table> 요소 찾은 것을 table에 저장하기
table = soup.find("div",{"class":"table_style01"}).find("table")

In [ ]:
data = []

# 첫번째 행부터 순회시키기
for tr in table.find_all("tr")[2:]:
  record = []
  # 종목 코드가 없는 경우 오류 발생 -> try except 구문으로 넘기는 작업 추가
  try:
    for i, td in enumerate(tr.find_all("td")[:6]): # 6번째 셀까지
      if i == 1:
        # remove_noise_title의 출려과 이어 붙이기

        record += remove_noise_title(td.text.split('\n\n')[1])
      # 노이즈가 있는 세번째 셀만 따로 처리
      elif i ==3 :
        record.append(td.text.replace(" ","").replace("\r","").replace("\n",""))
      # 1번째 셀이 아닌 경우
      else:
        record.append(td.text)  # <- 셀의 텍스트 값 그대로 입력

    data.append(record)
  except:
    pass

### Data -> DataFrame

In [ ]:
# df : 제목, 라벨링
samsung_down_1p = pd.DataFrame(data, columns = ["작성일","종목명","종목코드","제목","작성자",
                              "작성기관","현재목표","이전목표"])

In [ ]:
samsung_down_1p

,작성일,종목명,종목코드,제목,작성자,작성기관,현재목표,이전목표
0,2023-04-10,삼성전자,005930,1Q23 잠정실적 리뷰,김광진,한화투자증권,0,"82,000"
1,2023-04-10,삼성전자,005930,1Q23P Samsung Pivot 발생 그 이후,김선우,메리츠증권,0,"79,000"
2,2023-02-01,삼성전자,005930,좀더 구체화된 감산 계획,송명섭,하이투자증권,"75,000","80,000"
3,2023-01-09,삼성전자,005930,실적 부진은 Samsung Pivot 의 필요 조건,김선우,메리츠증권,0,"82,000"
4,2022-10-28,삼성전자,005930,메모리가 실적 부진 주도,김운호,IBK투자증권,"70,000","88,000"
...,...,...,...,...,...,...,...,...
75,2018-12-14,삼성전자,005930,3만원대 중후반이 주가 저점일 듯,송명섭,하이투자증권,"48,000","51,000"
76,2018-12-12,삼성전자,005930,1Q19 실적이 바닥입니다,이재윤,유안타증권,"52,000","76,000"
77,2018-12-03,삼성전자,005930,이익 추정 하향보다 무역갈등 완화에 주목,"김경민,이진우",하나금융투자,"54,000","57,000"
78,2018-11-26,삼성전자,005930,글로벌 경기 불확실성 해소를 기다리며,박유악,키움증권,"58,000","62,000"


In [ ]:
samsung_down_1p['제목'] = samsung_down_1p['종목명']+samsung_down_1p['제목']

### DataFrame 라벨링

In [ ]:
samsung_down_1p['label'] = 0

In [ ]:
samsung_down_1p.drop(labels=['작성자', '작성기관','현재목표','이전목표'], axis = 1,inplace=True)

In [ ]:
samsung_down_1p

,작성일,종목명,종목코드,제목,label
0,2023-04-10,삼성전자,005930,삼성전자 1Q23 잠정실적 리뷰,0
1,2023-04-10,삼성전자,005930,삼성전자 1Q23P Samsung Pivot 발생 그 이후,0
2,2023-02-01,삼성전자,005930,삼성전자 좀더 구체화된 감산 계획,0
3,2023-01-09,삼성전자,005930,삼성전자 실적 부진은 Samsung Pivot 의 필요 조건,0
4,2022-10-28,삼성전자,005930,삼성전자 메모리가 실적 부진 주도,0
...,...,...,...,...,...
75,2018-12-14,삼성전자,005930,삼성전자 3만원대 중후반이 주가 저점일 듯,0
76,2018-12-12,삼성전자,005930,삼성전자 1Q19 실적이 바닥입니다,0
77,2018-12-03,삼성전자,005930,삼성전자 이익 추정 하향보다 무역갈등 완화에 주목,0
78,2018-11-26,삼성전자,005930,삼성전자 글로벌 경기 불확실성 해소를 기다리며,0


## 2 Page

In [ ]:
### requests.get()로 페이지 열기 -> BeautifulSoup의 입력
url = "http://hkconsensus.hankyung.com/apps.analysis/analysis.list?skinType=stock_bad&sdate=2018-06-05&edate=2023-06-05&search_value=&search_text=%BB%EF%BC%BA%C0%FC%C0%DA&pagenum=80&order_type=&now_page=2"

# headers를 입력해야 데이터가 리턴됨
html = requests.get(url, headers={'User-Agent':'Gils'}).content
soup = BeautifulSoup(html, 'lxml')

In [ ]:
### soup의 find로 table_style01인 div 요소 찾기
'''
find() : 조건 만족하는 첫 번째 요소 반환하고, find_all함수는 모든 요소를 리스트 형태로 반환
'''

'\nfind() : 조건 만족하는 첫 번째 요소 반환하고, find_all함수는 모든 요소를 리스트 형태로 반환\n'

In [ ]:
### 출력값에서 <table> 요소 찾은 것을 table에 저장하기
table = soup.find("div",{"class":"table_style01"}).find("table")

In [ ]:
data = []

# 첫번째 행부터 순회시키기
for tr in table.find_all("tr")[2:]:
  record = []
  # 종목 코드가 없는 경우 오류 발생 -> try except 구문으로 넘기는 작업 추가
  try:
    for i, td in enumerate(tr.find_all("td")[:6]): # 6번째 셀까지
      if i == 1:
        # remove_noise_title의 출려과 이어 붙이기

        record += remove_noise_title(td.text.split('\n\n')[1])
      # 노이즈가 있는 세번째 셀만 따로 처리
      elif i ==3 :
        record.append(td.text.replace(" ","").replace("\r","").replace("\n",""))
      # 1번째 셀이 아닌 경우
      else:
        record.append(td.text)  # <- 셀의 텍스트 값 그대로 입력

    data.append(record)
  except:
    pass

### Data -> DataFrame

In [ ]:
# df : 제목, 라벨링
samsung_down_2p = pd.DataFrame(data, columns = ["작성일","종목명","종목코드","제목","작성자",
                              "작성기관","현재목표","이전목표"])

In [ ]:
samsung_down_2p

,작성일,종목명,종목코드,제목,작성자,작성기관,현재목표,이전목표
0,2018-11-22,삼성전자,005930,반도체업종의 대형주 최선호주로 유지,김경민,하나금융투자,"57,000","60,000"
1,2018-11-15,삼성전자,005930,2019년 상반기 저점 통과할 것,"김영우,안영진",SK증권,"60,000","70,000"
2,2018-11-05,삼성전자,005930,과도한 우려가 반영된 주가,"김동원,황고운",KB증권,"52,000","55,000"
3,2018-11-01,삼성전자,005930,2019년을 볼 시점,"도현우,서준현",NH투자증권,"60,000","70,000"
4,2018-11-01,삼성전자,005930,숨고르기,김양재,KTB투자증권,"58,000","65,000"
5,2018-11-01,삼성전자,005930,저점 확인 구간,이순학,한화투자증권,"56,000","66,000"
6,2018-11-01,삼성전자,005930,4Q18 영업이익 16 2조원 전망,송명섭,하이투자증권,"51,000","55,500"
7,2018-11-01,삼성전자,005930,중장기 수요에 대한 자신감,박유악,키움증권,"62,000","68,000"
8,2018-11-01,삼성전자,005930,주가 하락의 핑계를 찾기도 힘들다,어규진,이베스트증권,"62,000","68,000"
9,2018-11-01,삼성전자,005930,불확실한 매크로 환경 및 이익 감소 구간 진입,이승우,유진투자증권,"54,000","57,000"


In [ ]:
samsung_down_2p['제목'] = samsung_down_2p['종목명']+samsung_down_2p['제목']

### DataFrame 라벨링

In [ ]:
samsung_down_2p['label'] = 0

In [ ]:
samsung_down_2p.drop(labels=['작성자', '작성기관','현재목표','이전목표'], axis = 1,inplace=True)

In [ ]:
samsung_down_2p

,작성일,종목명,종목코드,제목,label
0,2018-11-22,삼성전자,005930,삼성전자 반도체업종의 대형주 최선호주로 유지,0
1,2018-11-15,삼성전자,005930,삼성전자 2019년 상반기 저점 통과할 것,0
2,2018-11-05,삼성전자,005930,삼성전자 과도한 우려가 반영된 주가,0
3,2018-11-01,삼성전자,005930,삼성전자 2019년을 볼 시점,0
4,2018-11-01,삼성전자,005930,삼성전자 숨고르기,0
5,2018-11-01,삼성전자,005930,삼성전자 저점 확인 구간,0
6,2018-11-01,삼성전자,005930,삼성전자 4Q18 영업이익 16 2조원 전망,0
7,2018-11-01,삼성전자,005930,삼성전자 중장기 수요에 대한 자신감,0
8,2018-11-01,삼성전자,005930,삼성전자 주가 하락의 핑계를 찾기도 힘들다,0
9,2018-11-01,삼성전자,005930,삼성전자 불확실한 매크로 환경 및 이익 감소 구간 진입,0


# **크롤링 데이터 결합 및 저장**

In [ ]:
print(samsung_up_1p.head(3))
print('-'*80)
print(samsung_up_2p.head(3))
print('-'*80)
print(samsung_down_1p.head(3))
print('-'*80)
print(samsung_down_2p.head(3))

          작성일   종목명    종목코드                       제목  label
0  2023-04-10  삼성전자  005930  삼성전자 업황이 나쁠수록 감산은 확대된다       1
1  2023-03-15  삼성전자  005930     삼성전자 뻔하지만 당연한 최고의 선택      1
2  2023-01-09  삼성전자  005930        삼성전자 4Q22 잠정실적 리뷰      1
--------------------------------------------------------------------------------
          작성일   종목명    종목코드                     제목  label
0  2020-01-06  삼성전자  005930       삼성전자 2020년 주가 전망      1
1  2020-01-02  삼성전자  005930       삼성전자 It s bliss       1
2  2020-01-02  삼성전자  005930  삼성전자 새로운 서막이 될 1분기 실적      1
--------------------------------------------------------------------------------
          작성일   종목명    종목코드                                 제목  label
0  2023-04-10  삼성전자  005930                  삼성전자 1Q23 잠정실적 리뷰      0
1  2023-04-10  삼성전자  005930  삼성전자 1Q23P Samsung Pivot 발생 그 이후       0
2  2023-02-01  삼성전자  005930                 삼성전자 좀더 구체화된 감산 계획      0
--------------------------------------------------------------------------------
     

In [ ]:
samsung_up = pd.concat([samsung_up_1p,samsung_up_2p],axis = 0,ignore_index=True)
samsung_up

,작성일,종목명,종목코드,제목,label
0,2023-04-10,삼성전자,005930,삼성전자 업황이 나쁠수록 감산은 확대된다,1
1,2023-03-15,삼성전자,005930,삼성전자 뻔하지만 당연한 최고의 선택,1
2,2023-01-09,삼성전자,005930,삼성전자 4Q22 잠정실적 리뷰,1
3,2022-11-21,삼성전자,005930,삼성전자 경쟁력 격차 복구 여부에 주목 저점 분할 매수 권고,1
4,2022-07-29,삼성전자,005930,삼성전자 시험은 어렵지만 변별력은 높아진다,1
...,...,...,...,...,...
122,2019-02-01,삼성전자,005930,삼성전자 데이터포인트가 가리키는 곳,1
123,2019-02-01,삼성전자,005930,삼성전자 2Q19 수요 회복 기대,1
124,2019-02-01,삼성전자,005930,삼성전자,1
125,2019-02-01,삼성전자,005930,삼성전자 어느새 비싸진 주가,1


In [ ]:
samsung_down = pd.concat([samsung_down_1p,samsung_down_2p],axis = 0,ignore_index=True)
samsung_down

,작성일,종목명,종목코드,제목,label
0,2023-04-10,삼성전자,005930,삼성전자 1Q23 잠정실적 리뷰,0
1,2023-04-10,삼성전자,005930,삼성전자 1Q23P Samsung Pivot 발생 그 이후,0
2,2023-02-01,삼성전자,005930,삼성전자 좀더 구체화된 감산 계획,0
3,2023-01-09,삼성전자,005930,삼성전자 실적 부진은 Samsung Pivot 의 필요 조건,0
4,2022-10-28,삼성전자,005930,삼성전자 메모리가 실적 부진 주도,0
...,...,...,...,...,...
105,2018-06-25,삼성전자,005930,삼성전자 2Q18 Preview 숨고르기,0
106,2018-06-20,삼성전자,005930,삼성전자 시선을 3분기로,0
107,2018-06-20,삼성전자,005930,삼성전자 신규 아이폰은 여전히 기회다,0
108,2018-06-11,삼성전자,005930,삼성전자 전자도 어쩔 수 없는 스마트폰 시장,0


In [ ]:
# 상향 하향 결합
samsung = pd.concat([samsung_up,samsung_down],axis = 0,ignore_index=True)
samsung

,작성일,종목명,종목코드,제목,label
0,2023-04-10,삼성전자,005930,삼성전자 업황이 나쁠수록 감산은 확대된다,1
1,2023-03-15,삼성전자,005930,삼성전자 뻔하지만 당연한 최고의 선택,1
2,2023-01-09,삼성전자,005930,삼성전자 4Q22 잠정실적 리뷰,1
3,2022-11-21,삼성전자,005930,삼성전자 경쟁력 격차 복구 여부에 주목 저점 분할 매수 권고,1
4,2022-07-29,삼성전자,005930,삼성전자 시험은 어렵지만 변별력은 높아진다,1
...,...,...,...,...,...
232,2018-06-25,삼성전자,005930,삼성전자 2Q18 Preview 숨고르기,0
233,2018-06-20,삼성전자,005930,삼성전자 시선을 3분기로,0
234,2018-06-20,삼성전자,005930,삼성전자 신규 아이폰은 여전히 기회다,0
235,2018-06-11,삼성전자,005930,삼성전자 전자도 어쩔 수 없는 스마트폰 시장,0


In [ ]:
samsung.to_csv('한경_삼성전자_이정한.csv')

In [ ]:
# 확인
data = pd.read_csv("한경_삼성전자_이정한.csv", sep=",",index_col = 0)
data

,작성일,종목명,종목코드,제목,label
0,2023-04-10,삼성전자,005930,삼성전자 업황이 나쁠수록 감산은 확대된다,1
1,2023-03-15,삼성전자,005930,삼성전자 뻔하지만 당연한 최고의 선택,1
2,2023-01-09,삼성전자,005930,삼성전자 4Q22 잠정실적 리뷰,1
3,2022-11-21,삼성전자,005930,삼성전자 경쟁력 격차 복구 여부에 주목 저점 분할 매수 권고,1
4,2022-07-29,삼성전자,005930,삼성전자 시험은 어렵지만 변별력은 높아진다,1
...,...,...,...,...,...
232,2018-06-25,삼성전자,005930,삼성전자 2Q18 Preview 숨고르기,0
233,2018-06-20,삼성전자,005930,삼성전자 시선을 3분기로,0
234,2018-06-20,삼성전자,005930,삼성전자 신규 아이폰은 여전히 기회다,0
235,2018-06-11,삼성전자,005930,삼성전자 전자도 어쩔 수 없는 스마트폰 시장,0
